# Building a Customer Support Agent with Cortex AI at Tasty Bytes

**Tasty Bytes** is a fictional global food truck enterprise that has established its presence in 30 cities across 15 countries, boasting a network of 450 trucks offering 15 diverse menu types under various brands. Our mission at Tasty Bytes is to improve Customer Experiences by leveraging the power of AI with Snowflake Cortex.

## Power of Snowflake Cortex LLM Fine Tuning
In this tutorial, we will build an LLM-based customer support agent that showcases the power of Cortex Fine-Tuning and helps the Tasty Bytes team respond with highly accurate automated emails to customer tickets, all with minimal resources and time. Fine-tuning has significantly advanced the Tasty Bytes team’s ability to meet the key objective of enhancing customer experiences.



In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()


In [ ]:
SHOW TABLES;

The `SUPPORT_EMAILS` table contains parsed email data from customers, which is essential for building and fine-tuning the customer support agent. This table includes the following columns:

- **subject**: The subject line of the email, providing a brief summary of the customer's query or issue.
- **body**: The main content of the email, containing the detailed message from the customer.
- **timestamp**: The date and time when the email was received, allowing for tracking and analyzing the timing of customer interactions.
- **labeled_location**: The location of the truck,this is detected from the message or human in the flow can append this
- **labeled_truck**: The name of the truck,this is detected from the message or human in the flow can append this
- **response** : The actual response that will be used to construct prompt and response to train the model


In [ ]:
SELECT * FROM SUPPORT_EMAILS LIMIT 2;

* Build the Training table and a Validation table based on the Support_Emails table.This is already created and shared for your convenience using a Mod function on the Id field so that the Train and Test split can be built that will give a reproducible sample.
* A field called golden JSON was created. This contains the location,truck and the response body.
* The body will be used as a prompt and the Golden JSON will be the completion for training the LLM.

In [ ]:
CREATE OR REPLACE FUNCTION BUILD_EXAMPLE(location STRING, truck STRING, response_body STRING)
RETURNS STRING
LANGUAGE SQL
AS
$$
CONCAT(
'{ location: ', IFF(location IS NULL, 'null', CONCAT('"', location, '"')), 
', truck: ', IFF(truck IS NULL, 'null', CONCAT('"', truck, '"')), 
', response_body: ', IFF(response_body IS NULL, 'null', CONCAT('"', REPLACE(response_body, '\n', '\\n'), '"')), '
}')
$$
;

* Split 80% training data and 20% validation data


In [ ]:
CREATE OR REPLACE TABLE VALIDATION AS (SELECT *, BUILD_EXAMPLE(LABELED_LOCATION, LABELED_TRUCK, SUPPORT_RESPONSE) as GOLDEN_JSON FROM SUPPORT_EMAILS 
-- Split: 20% validation data, 80% training data
WHERE ID % 10 < 2);

In [ ]:
CREATE OR REPLACE TABLE TRAINING AS (SELECT *,BUILD_EXAMPLE(LABELED_LOCATION, LABELED_TRUCK, SUPPORT_RESPONSE) as GOLDEN_JSON FROM SUPPORT_EMAILS WHERE ID % 10 >= 2);

In [ ]:
SELECT * FROM TRAINING LIMIT 2;

In [ ]:
SELECT * FROM VALIDATION LIMIT 2;

* PROMPT ENGINEERING 

Create a Prompt engineering template that takes in the customer email as input and with instructing fine tuning, generates a response based on presence of required annotations

In [ ]:

CREATE OR REPLACE FUNCTION PROMPT_TEMPLATE_EXTRACT_JSON(customer_message STRING)
RETURNS STRING
LANGUAGE SQL
AS
$$
CONCAT(
'You are a customer support agent of a food truck franchise company TastyBytes. 

Customers will send messages with feedback on food trucks, their experience with 
the food they ordered or bought or ask questions about the menu. 

In order to respond to the messages, you need to know the name of the food truck 
and the city the food truck is located in. If the customer message does not 
contain the name of the food truck or the city, you have to send a follow-up 
message to the customer asking for that information. If the city is missing, do 
not ask the customer where the truck is located but instead ask in which city 
they experienced the issue. If the name of the truck name is missing, please 
kindly ask the customer to clarify which truck they are referring to since there 
are multiple TastyBytes trucks in the city.

In your message, you should acknowledge and reference the customer\'s message 
content. Please thank the customer for their message. If the customer complains, 
please apologize to the customer.

If the customer is asking for a refund, please state that you will issue the 
refund and that there will be a separate message to enter the payment details. 

Do respond as if you were a human. Do not mention that you are an AI assistant. 
Do not ask the customer to contact anyone else.

Customer message: ', customer_message,'

Format your response as valid JSON with the fields:
"location": the location of the truck mentioned in the customer email, null if missing;
"truck": the name of the truck mentioned in the customer email, null if missing;
"response_body": contents of your response to the customer.

')
$$
;


* Create a table that will contain the output from applying the function with the prompt engineering instruction tuning template

In [ ]:
CREATE OR REPLACE TABLE FINE_TUNING_VALIDATION_BASE_PROMPTING AS (
    SELECT
        -- Carry over fields from source for convenience.
        ID, BODY, LABELED_LOCATION, LABELED_TRUCK, 
        -- Run the LLM with the prompt template from above.
        SNOWFLAKE.CORTEX.COMPLETE(
            -- Base model
            'mistral-7b',
            -- Prompt template
            PROMPT_TEMPLATE_EXTRACT_JSON(body)
        ) AS RESPONSE
    FROM VALIDATION
);

In [ ]:
select RESPONSE FROM FINE_TUNING_VALIDATION_BASE_PROMPTING LIMIT 2;

In [ ]:

SELECT
    LABELED_LOCATION,
    TRY_PARSE_JSON(RESPONSE):location::STRING as BASE_MODEL_RESPONSE_LOCATION
FROM FINE_TUNING_VALIDATION_BASE_PROMPTING;



In [ ]:
SELECT
    LABELED_TRUCK,
    TRY_PARSE_JSON(RESPONSE):truck::STRING as BASE_MODEL_RESPONSE_TRUCK,
FROM FINE_TUNING_VALIDATION_BASE_PROMPTING;

* Determine Accuracy of the output from Prompt Engineered Response

In [ ]:
CREATE OR REPLACE FUNCTION ACCURACY(candidate STRING, reference STRING)
RETURNS number
LANGUAGE SQL
AS
$$
DIV0(SUM(IFF(
    EQUAL_NULL(
        reference, 
        candidate
    ),
    -- THEN
    1,
    -- ELSE
    0
)), COUNT(*))
$$
;

In [ ]:
SELECT
    ACCURACY(TRY_PARSE_JSON(RESPONSE):location::STRING, LABELED_LOCATION) AS BASE_MODEL_ACCURACY_LOCATION,
    ACCURACY(TRY_PARSE_JSON(RESPONSE):truck::STRING, LABELED_TRUCK) AS BASE_MODEL_ACCURACY_TRUCK
FROM FINE_TUNING_VALIDATION_BASE_PROMPTING;

## CORTEX FINETUNING 
Cortex Fine-Tuning is a fully managed service that lets you fine-tune popular LLMs using your data, all within Snowflake.

Call the SNOWFLAKE.CORTEX.FINETUNE function and pass in ‘CREATE’ as the first argument to start a fine-tuning job. Specify which base model needs to be used. 

For entire list of models available to fine tune refer to Snowflake Documentation 
https://docs.snowflake.com/en/user-guide/snowflake-cortex/cortex-finetuning

In [ ]:
SELECT SNOWFLAKE.CORTEX.FINETUNE(
    'CREATE', 
    -- Custom model name, make sure name below is unique
    'SUPPORT_MISTRAL_7B',
    -- Base model name
    'mistral-7b',
    -- Training data query
    'SELECT BODY AS PROMPT, GOLDEN_JSON AS COMPLETION FROM TRAINING',
    -- Validation data query 
    'SELECT BODY AS PROMPT, GOLDEN_JSON AS COMPLETION FROM VALIDATION' 
);

## Wait for ~5-10 mins for the job to complete. If the Show Models in the below displays the model specified in above Fine tune creation then that denotes successful creation of the model.

In [ ]:
--Pick the output of last cell's result and add it below replacing the ''
Select SNOWFLAKE.CORTEX.FINETUNE(
  'DESCRIBE',
  ''
);

#### List the fine-tuning jobs for which you have access privileges.

In [ ]:
SHOW MODELS;

In [ ]:
CREATE OR REPLACE TABLE FINE_TUNING_VALIDATION_FINETUNED AS (
    SELECT
        -- Carry over fields from source for convenience.
        ID, BODY, LABELED_TRUCK, LABELED_LOCATION,
        -- Run the custom fine-tuned LLM.
        SNOWFLAKE.CORTEX.COMPLETE(
            -- Custom model
            'SUPPORT_MISTRAL_7B', 
            -- No prompt template needed.
            body 
        ) AS RESPONSE
    FROM VALIDATION
);


In [ ]:
SELECT RESPONSE FROM FINE_TUNING_VALIDATION_FINETUNED LIMIT 5;

In [ ]:
SELECT
    LABELED_LOCATION,
    TRY_PARSE_JSON(RESPONSE):location::STRING as FINETUNED_MODEL_RESPONSE_LOCATION,
FROM FINE_TUNING_VALIDATION_FINETUNED;

In [ ]:
SELECT
    LABELED_TRUCK,
    TRY_PARSE_JSON(RESPONSE):truck::STRING as FINETUNED_MODEL_RESPONSE_TRUCK,
FROM FINE_TUNING_VALIDATION_FINETUNED;

## SCORING

Let's analyze the difference between carrying just Prompt engineering vs fine tuning a LLM using Cortex

In [ ]:

SELECT
    ACCURACY(TRY_PARSE_JSON(RESPONSE):location::STRING, LABELED_LOCATION) AS ACCURACY_LOCATION,
    ACCURACY(TRY_PARSE_JSON(RESPONSE):truck::STRING, LABELED_TRUCK) AS ACCURACY_TRUCK
FROM FINE_TUNING_VALIDATION_FINETUNED;

In [ ]:
SELECT 
    base.LABELED_LOCATION, 
    TRY_PARSE_JSON(base.RESPONSE):location::STRING as BASE_MODEL_LOCATION,
    TRY_PARSE_JSON(custom.RESPONSE):location::STRING as FINETUNED_MODEL_LOCATION,
FROM FINE_TUNING_VALIDATION_BASE_PROMPTING as base
JOIN FINE_TUNING_VALIDATION_FINETUNED as custom
ON base.ID = custom.ID;

In [ ]:
SELECT 
    base.LABELED_TRUCK, 
    TRY_PARSE_JSON(base.RESPONSE):truck::STRING as BASE_MODEL_TRUCK,
    TRY_PARSE_JSON(custom.RESPONSE):truck::STRING as FINETUNED_MODEL_TRUCK,
FROM FINE_TUNING_VALIDATION_BASE_PROMPTING as base
JOIN FINE_TUNING_VALIDATION_FINETUNED as custom
ON base.ID = custom.ID;

## END OF THE DEMO